In [1]:
from mainnet_launch.abis.abis import CHAINLINK_KEEPER_REGISTRY_ABI
import pandas as pd
from mainnet_launch.constants import (
    CACHE_TIME,
    eth_client,
    ALL_AUTOPOOLS,
    BAL_ETH,
    AUTO_ETH,
    AUTO_LRT,
    AutopoolConstants,
)
from concurrent.futures import ThreadPoolExecutor
from mainnet_launch.data_fetching.get_events import fetch_events
import threading
import numpy as np

from mainnet_launch.data_fetching.get_state_by_block import (
    get_raw_state_by_blocks,
    get_state_by_one_block,
    add_timestamp_to_df_with_block_column,
)
from mainnet_launch.solver_diagnostics.fetch_rebalance_events import _calc_gas_used_by_transaction_in_eth

topic_2 = "1344461886831441856282597505993515040672606510446374000438363195934269203116"
contract = eth_client.eth.contract("0x6593c7De001fC8542bB1703532EE1E5aA0D458fD", abi=CHAINLINK_KEEPER_REGISTRY_ABI)
start_block = 20500000  # Sep 15, 2024


upkeep_df = fetch_events(contract.events.UpkeepPerformed, start_block)
our_upkeep_df = upkeep_df[upkeep_df["id"].apply(str) == str(topic_2)].copy()
our_upkeep_df = add_timestamp_to_df_with_block_column(our_upkeep_df)  # 20 seoncds
our_upkeep_df

2024-10-12 16:26:42.948 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2024-10-12 16:26:43.018 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2024-10-12 16:26:43.019 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2024-10-12 16:26:43.020 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager


,id,success,totalPayment,gasUsed,gasOverhead,trigger,event,block,transaction_index,log_index,hash
timestamp,,,,,,,,,,,
2024-08-31 13:32:59,1344461886831441856282597505993515040672606510...,True,1222318964499686881,3155118,114740,b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00...,UpkeepPerformed,20648872,41,137,0x2b980cecd54fad7d5530d4d311077850da2f5e55a562...
2024-08-31 13:33:23,1344461886831441856282597505993515040672606510...,True,1228774339823369983,3327179,114740,b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00...,UpkeepPerformed,20648874,54,177,0x50979f1c4a4edc757c1f4c6f7b602481469fc9c34e69...
2024-08-31 13:42:47,1344461886831441856282597505993515040672606510...,True,1313831153875426419,3411278,114740,b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00...,UpkeepPerformed,20648921,52,132,0x12c00b710da71d53aa9bda66d849296c6ae558f8db0a...
2024-08-31 13:43:35,1344461886831441856282597505993515040672606510...,True,794810869676295344,2188017,114066,b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00...,UpkeepPerformed,20648925,38,72,0x67dc2c2f0d2f2ce9e61c7972a0a5f787f146d798ac4a...
2024-08-31 14:33:11,1344461886831441856282597505993515040672606510...,True,1155328259017426672,2640262,114740,b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00...,UpkeepPerformed,20649171,90,181,0xda1c2c7ab5d06560f35b4e50ef2c63b175f98442d4ef...
...,...,...,...,...,...,...,...,...,...,...,...
2024-10-12 22:10:59,1344461886831441856282597505993515040672606510...,True,4244673641660228827,1654812,112043,b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00...,UpkeepPerformed,20952417,5,38,0xe15f4b1dbdd954f02278f147cde952abbfde72193c15...
2024-10-12 22:40:35,1344461886831441856282597505993515040672606510...,True,1470903321240593667,499888,112043,b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00...,UpkeepPerformed,20952564,15,177,0xcf66e07532eaf21c7295ab4a130e66928b279ae15369...
2024-10-12 23:07:23,1344461886831441856282597505993515040672606510...,True,1651343969862889345,665704,112043,b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00...,UpkeepPerformed,20952697,10,66,0x2b537dc791cf7c320fa3a4d58233b42afbdf3bb9cdd2...


In [2]:
hash_to_gas_cost = {}
hash_to_gasPrice = {}
lock = threading.Lock()


def batch_calc_gas_used_by_transaction_in_eth(tx_hashes):
    for tx_hash in tx_hashes:
        tx_receipt = eth_client.eth.get_transaction_receipt(tx_hash)
        tx = eth_client.eth.get_transaction(tx_hash)
        gas_cost = float(eth_client.fromWei(tx["gasPrice"] * tx_receipt["gasUsed"], "ether"))
        with lock:
            hash_to_gas_cost[tx_hash] = gas_cost
            hash_to_gasPrice[tx_hash] = tx["gasPrice"]


with ThreadPoolExecutor(max_workers=25) as executor:
    hash_groups = np.array_split(our_upkeep_df["hash"], 100)
    for tx_hashes in hash_groups:
        executor.submit(batch_calc_gas_used_by_transaction_in_eth, tx_hashes)

our_upkeep_df["gasCostInETH"] = our_upkeep_df["hash"].apply(lambda x: hash_to_gas_cost[x])
our_upkeep_df["gasPrice"] = our_upkeep_df["hash"].apply(lambda x: hash_to_gasPrice[x])
our_upkeep_df["gasCostInETH_with_premium"] = our_upkeep_df["gasCostInETH"] * 1.2  # it costs a 20% premium
our_upkeep_df.to_csv("our_upkeep_df_with_gas_costs.csv")
our_upkeep_df

/home/parker/Documents/Tokemak/v2-rebalance-dashboard/.venv/lib/python3.10/site-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'Series.swapaxes' is deprecated and will be removed in a future version. Please use 'Series.transpose' instead.
  return bound(*args, **kwds)


,id,success,totalPayment,gasUsed,gasOverhead,trigger,event,block,transaction_index,log_index,hash,gasCostInETH,gasPrice,gasCostInETH_with_premium
timestamp,,,,,,,,,,,,,,
2024-08-31 13:32:59,1344461886831441856282597505993515040672606510...,True,1222318964499686881,3155118,114740,b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00...,UpkeepPerformed,20648872,41,137,0x2b980cecd54fad7d5530d4d311077850da2f5e55a562...,0.004516,1383318346,0.005419
2024-08-31 13:33:23,1344461886831441856282597505993515040672606510...,True,1228774339823369983,3327179,114740,b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00...,UpkeepPerformed,20648874,54,177,0x50979f1c4a4edc757c1f4c6f7b602481469fc9c34e69...,0.004539,1320718525,0.005447
2024-08-31 13:42:47,1344461886831441856282597505993515040672606510...,True,1313831153875426419,3411278,114740,b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00...,UpkeepPerformed,20648921,52,132,0x12c00b710da71d53aa9bda66d849296c6ae558f8db0a...,0.004853,1378275206,0.005823
2024-08-31 13:43:35,1344461886831441856282597505993515040672606510...,True,794810869676295344,2188017,114066,b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00...,UpkeepPerformed,20648925,38,72,0x67dc2c2f0d2f2ce9e61c7972a0a5f787f146d798ac4a...,0.002942,1280746859,0.003530
2024-08-31 14:33:11,1344461886831441856282597505993515040672606510...,True,1155328259017426672,2640262,114740,b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00...,UpkeepPerformed,20649171,90,181,0xda1c2c7ab5d06560f35b4e50ef2c63b175f98442d4ef...,0.004183,1553558719,0.005019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-10-12 22:10:59,1344461886831441856282597505993515040672606510...,True,4244673641660228827,1654812,112043,b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00...,UpkeepPerformed,20952417,5,38,0xe15f4b1dbdd954f02278f147cde952abbfde72193c15...,0.015956,9053970998,0.019147
2024-10-12 22:40:35,1344461886831441856282597505993515040672606510...,True,1470903321240593667,499888,112043,b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00...,UpkeepPerformed,20952564,15,177,0xcf66e07532eaf21c7295ab4a130e66928b279ae15369...,0.005571,9245328930,0.006686
2024-10-12 23:07:23,1344461886831441856282597505993515040672606510...,True,1651343969862889345,665704,112043,b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00...,UpkeepPerformed,20952697,10,66,0x2b537dc791cf7c320fa3a4d58233b42afbdf3bb9cdd2...,0.006272,8112134400,0.007527


In [11]:
px.line(our_upkeep_df.resample("1D")["gasPrice"].median())

In [14]:
daily_gas_price = our_upkeep_df.groupby(our_upkeep_df.index.date)["gasPrice"]

# Convert to DataFrame for easier plotting
daily_gas_price_df = daily_gas_price.apply(list).reset_index()
daily_gas_price_df.columns = ["Date", "GasPrices"]

# Explode the list of gas prices for each day to separate rows for box plot compatibility
exploded_df = daily_gas_price_df.explode("GasPrices")
exploded_df["GasPrices"] = exploded_df["GasPrices"].astype(float)  # Ensure data is numeric

# Generate a box plot in Plotly
fig = px.box(exploded_df, x="Date", y="GasPrices", title="Daily Gas Price Box Plot")

# Show the plot
fig.show()

In [8]:
px.histogram(our_upkeep_df["gasPrice"], title="gas price")

In [3]:
# our costs are the cost to execute the treansaction + a 20% premium.

# we pay this in LINK so there can be price movements in our favor or not in our favor ignoring that for now

In [4]:
our_upkeep_df["gasCostInETH_with_premium"].sum()  # we've spent 35 ETH since launch on gas costs.

np.float64(35.027575457039255)

In [5]:
total_daily_calculator_cost = our_upkeep_df.resample("1D")[["gasCostInETH_with_premium"]].sum()
total_daily_calculator_cost["cumulative_daily_cost"] = total_daily_calculator_cost["gasCostInETH_with_premium"].cumsum()
import plotly.express as px

px.line(total_daily_calculator_cost, title="daily and cumulative ETH gas costs")

In [6]:
px.bar(total_daily_calculator_cost["gasCostInETH_with_premium"])